# Script for preprocess of corpus in CSL dataset

In [2]:
f = open('dictionary.txt',encoding='utf-8')
words = f.readlines()

## Build isl dictionary

In [31]:
isl_dictionary = {}
for word in words:
    data = word.rstrip('\n').split()
    index = int(data[0])
    token = data[1]
    isl_dictionary[token] = index
def reverse_dictionary(dictionary):
    reverse_dict = {}
    for k,v in dictionary.items():
        reverse_dict[v] = k
    return reverse_dict
isl_dictionary['商人']

282

目的


## Build csl dictionary and reverse dictionary

In [21]:
import jieba
from collections import Counter
def build_csl_dictionary():
    annotation_file = open("corpus.txt",'r')
    corpus = annotation_file.readlines()
    corpus = [sentence.rstrip('\n').split()[1] for sentence in corpus]
    # Create a dictionary which maps tokens to indices (train contains all the training sentences)
    freq_list = Counter()
    punctuation = ['\ufeff']
    for sentence in corpus:
        sentence = [word for word in jieba.cut(sentence) if not word in punctuation]
        freq_list.update(sentence)

    # 按照词的出现频率建立词典，词频越高索引越靠前
    freq_list = sorted(freq_list.items(),key=lambda item:item[1],reverse=True)
    dictionary = {}
    dictionary['<pad>'] = 0
    dictionary['<bos>'] = 1
    dictionary['<eos>'] = 2
    for i,item in enumerate(freq_list):
        dictionary[item[0]] = i+3
    print("Build CSL dictionary successfully!")
    return dictionary
dictionary = build_csl_dictionary()
reverse_dict = reverse_dictionary(dictionary)

Build CSL dictionary successfully!


In [27]:
reverse_dict[6]

{'<pad>': 0,
 '<bos>': 1,
 '<eos>': 2,
 '的': 3,
 '是': 4,
 '他': 5,
 '我': 6,
 '你': 7,
 '我们': 8,
 '同学': 9,
 '社会': 10,
 '有': 11,
 '工作': 12,
 '国家': 13,
 '妈妈': 14,
 '哥哥': 15,
 '外祖父': 16,
 '爸爸': 17,
 '目标': 18,
 '朋友': 19,
 '祖父': 20,
 '妹妹': 21,
 '丈夫': 22,
 '新': 23,
 '提高': 24,
 '地位': 25,
 '稳定': 26,
 '基础': 27,
 '幸福': 28,
 '警察': 29,
 '护士': 30,
 '她': 31,
 '邻居': 32,
 '残疾人': 33,
 '公公': 34,
 '婆婆': 35,
 '儿子': 36,
 '裁缝': 37,
 '紧张': 38,
 '改善': 39,
 '民主': 40,
 '团结': 41,
 '毛毯': 42,
 '婚姻': 43,
 '形势': 44,
 '困难': 45,
 '多': 46,
 '地球': 47,
 '情况': 48,
 '好': 49,
 '就业': 50,
 '公务员': 51,
 '商人': 52,
 '解放军': 53,
 '姐姐': 54,
 '模特': 55,
 '工人': 56,
 '保姆': 57,
 '妻子': 58,
 '教师': 59,
 '导演': 60,
 '演员': 61,
 '女朋友': 62,
 '教练': 63,
 '医生': 64,
 '律师': 65,
 '外祖母': 66,
 '园丁': 67,
 '炊事员': 68,
 '表哥': 69,
 '记者': 70,
 '牧民': 71,
 '保育员': 72,
 '弱智人': 73,
 '嫂嫂': 74,
 '画家': 75,
 '知识分子': 76,
 '猎手': 77,
 '保安': 78,
 '邮递员': 79,
 '祖母': 80,
 '聋人': 81,
 '门卫': 82,
 '会计': 83,
 '武警': 84,
 '盲人': 85,
 '编辑': 86,
 '农民': 87,
 '职员': 88,
 '弟弟': 89,
 '向导': 90

In [ ]:
import jieba
f = open('corpus.txt','r')
sentences = f.readlines()
count = 0
csl_dictionary = {}
punctuation = [' ','\n','\ufeff']
for sentence in sentences:
    words = jieba.cut(sentence.rstrip('\n'))
    for word in words:
        if word not in csl_dictionary.values() and word not in punctuation and '0' not in word:
            csl_dictionary[word] = count
            count += 1

In [ ]:
def isIndict(k,dictionary):
    for word in dictionary.keys():
        if k in word:
            return word
    return -1
isl_in_csl_dictionary = {}
for k in csl_dictionary.keys():
    word = isIndict(k,isl_dictionary)
    if word!=-1:
        index = isl_dictionary[word]
        isl_in_csl_dictionary[k] = index
    else:
        print(k)
# isl_in_csl_dictionary = sorted(isl_in_csl_dictionary.items(),key=lambda item:item[1])

In [ ]:
# Generate subset file for validation
subset_index_list = [record[1] for record in isl_in_csl_dictionary]

import os
import os.path as osp

def create_path(path):
    if not osp.exists(path):
        os.makedirs(path)

num_class = 500
color_video_root = "/home/liweijie/SLR_dataset/S500_color_video"
skeleton_root = "/home/liweijie/SLR_dataset/xf500_body_color_txt"
val_list = open("../input/subset_val_list.txt","w")

color_video_path_list = os.listdir(color_video_root)
color_video_path_list.sort()
n = len(color_video_path_list)
for i,color_video_path in enumerate(color_video_path_list):
    print("%d/%d"%(i,n))
    label = color_video_path
    abs_color_video_path = osp.join(color_video_root,color_video_path)
    color_video_list = os.listdir(abs_color_video_path)
    color_video_list.sort()
    index = int(label)
    if index in subset_index_list:
        for color_video in color_video_list:
            abs_color_video = osp.join(abs_color_video_path,color_video)
            if(osp.isdir(abs_color_video)):
                p = color_video.split('_')
                person = int(p[0].lstrip('P'))
                num_frames = len(os.listdir(abs_color_video))
                path = osp.join(color_video_path,color_video)
                if not '(' in path:
                    path_skeleton = path.rstrip("color")+"body.txt"
                    abs_path_skeleton = osp.join(skeleton_root,path_skeleton)
                    if osp.exists(abs_path_skeleton):
                        record = path+"\t"+path_skeleton+"\t"+\
                                            str(num_frames)+"\t"+color_video_path+"\n"
                        val_list.write(record)
                    else:
                        print("The skeleton path %s don't exist"%abs_path_skeleton)

In [ ]:
reverse_dict[4]

In [ ]:
isl_dictionary